In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

We will use keras tuner to hyper tune the models

In [6]:
!pip install -q -U keras-tuner

In [7]:
import keras_tuner as kt
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers

Copying the dataset

In [8]:
df = pd.read_csv('data/simpleData.csv')

In [9]:
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

In [10]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('price')
test_labels = test_features.pop('price')

In [11]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

Defining a model

In [12]:
def build_and_compile_model(hp):
  model = keras.Sequential([
      normalizer,
      layers.Dense(units=hp.Int("units_l1", 16, 512, step=16),
                          use_bias=hp.Boolean("bias_l1"),
                          activation=hp.Choice("act_l1", ["relu", "tanh"])),
      layers.Dense(units=hp.Int("units_l2", 16, 512, step=16), 
                          use_bias=hp.Boolean("bias_l2"),
                          activation=hp.Choice("act_l2", ["relu", "tanh"])),
      layers.Dense(units=hp.Int("units_l3", 16, 512, step=16),
                          use_bias=hp.Boolean("bias_l3"),
                          activation=hp.Choice("act_l3", ["relu", "tanh"])),
      layers.Dense(1)
  ])

  optim=hp.Choice("optimizer",["sgd","rmsprop","adam"])
  
  model.compile(optimizer=optim,
                loss="mean_squared_error", metrics=["mean_squared_error"])
  return model

We are using a hyperband tuner and also adding a call back tp stop training after a certain validation loss

In [13]:
tuner = kt.Hyperband(build_and_compile_model,
                    objective=kt.Objective(name="val_loss",direction="min"),
                     max_epochs=10,
                     #max_trials=3,
                     #executions_per_trial=2,
                     factor=3,
                     directory='models',
                     project_name='ride_model_param_tuning')

In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=5)

In [15]:
tuner.search(train_features, train_labels, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 01m 18s]
val_loss: 7.399877071380615

Best val_loss So Far: 7.377449989318848
Total elapsed time: 00h 25m 04s
INFO:tensorflow:Oracle triggered exit


In [16]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]

print(f"""
The hyperparameter search is complete. The optimizer of units in the densely-connected
layer1 {best_hps.get('units_l1')}, activation {best_hps.get('act_l1')} and bias {best_hps.get('bias_l1')}
layer2 {best_hps.get('units_l2')}, activation {best_hps.get('act_l2')} and bias {best_hps.get('bias_l2')}
layer3 {best_hps.get('units_l3')}, activation {best_hps.get('act_l3')} and bias {best_hps.get('bias_l3')}
is {best_hps.get('optimizer')}.
""")


The hyperparameter search is complete. The optimizer of units in the densely-connected
layer1 416, activation relu and bias False
layer2 192, activation tanh and bias False
layer3 96, activation tanh and bias False
is adam.



We can use the results from tuning the parameters to build the model

In [21]:
def build_model(normalizer):
  model = keras.Sequential([
      normalizer,
      layers.Dense(units=416,
                          use_bias=False,
                          activation="relu"),
      layers.Dense(units=192,
                          use_bias=False,
                          activation="tanh"),
      layers.Dense(units=96,
                          use_bias=False,
                          activation="tanh"),
      layers.Dense(1)
  ])  
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.0001))
  return model

In [22]:
dnn_model = build_model(normalizer)

In [23]:
dnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 5)                11        
 n)                                                              
                                                                 
 dense_8 (Dense)             (None, 416)               2080      
                                                                 
 dense_9 (Dense)             (None, 192)               79872     
                                                                 
 dense_10 (Dense)            (None, 96)                18432     
                                                                 
 dense_11 (Dense)            (None, 1)                 97        
                                                                 
Total params: 100,492
Trainable params: 100,481
Non-trainable params: 11
_______________________________________________

In [24]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=2, epochs=200)

Epoch 1/200
1925/1925 - 11s - loss: 5.7140 - val_loss: 4.6919 - 11s/epoch - 6ms/step
Epoch 2/200
1925/1925 - 5s - loss: 4.5342 - val_loss: 4.5161 - 5s/epoch - 3ms/step
Epoch 3/200
1925/1925 - 5s - loss: 4.2450 - val_loss: 4.1278 - 5s/epoch - 3ms/step
Epoch 4/200
1925/1925 - 5s - loss: 3.7699 - val_loss: 3.4422 - 5s/epoch - 3ms/step
Epoch 5/200
1925/1925 - 5s - loss: 2.8245 - val_loss: 2.3695 - 5s/epoch - 3ms/step
Epoch 6/200
1925/1925 - 5s - loss: 2.0853 - val_loss: 1.9508 - 5s/epoch - 3ms/step
Epoch 7/200
1925/1925 - 5s - loss: 1.8018 - val_loss: 1.7599 - 5s/epoch - 3ms/step
Epoch 8/200
1925/1925 - 5s - loss: 1.6716 - val_loss: 1.7112 - 5s/epoch - 3ms/step
Epoch 9/200
1925/1925 - 5s - loss: 1.6127 - val_loss: 1.6254 - 5s/epoch - 3ms/step
Epoch 10/200
1925/1925 - 5s - loss: 1.5711 - val_loss: 1.6135 - 5s/epoch - 3ms/step
Epoch 11/200
1925/1925 - 5s - loss: 1.5399 - val_loss: 1.5700 - 5s/epoch - 3ms/step
Epoch 12/200
1925/1925 - 5s - loss: 1.5169 - val_loss: 1.5582 - 5s/epoch - 3ms/step

In [25]:
dnn_model.save('models/trained_model')

INFO:tensorflow:Assets written to: models/trained_model/assets


In [26]:
!zip -r models.zip models

  adding: models/ (stored 0%)
  adding: models/ride_model_param_tuning/ (stored 0%)
  adding: models/ride_model_param_tuning/trial_0017/ (stored 0%)
  adding: models/ride_model_param_tuning/trial_0017/trial.json (deflated 77%)
  adding: models/ride_model_param_tuning/trial_0017/checkpoint.data-00000-of-00001 (deflated 18%)
  adding: models/ride_model_param_tuning/trial_0017/checkpoint.index (deflated 61%)
  adding: models/ride_model_param_tuning/trial_0017/checkpoint (deflated 48%)
  adding: models/ride_model_param_tuning/trial_0012/ (stored 0%)
  adding: models/ride_model_param_tuning/trial_0012/trial.json (deflated 77%)
  adding: models/ride_model_param_tuning/trial_0012/checkpoint.data-00000-of-00001 (deflated 6%)
  adding: models/ride_model_param_tuning/trial_0012/checkpoint.index (deflated 59%)
  adding: models/ride_model_param_tuning/trial_0012/checkpoint (deflated 48%)
  adding: models/ride_model_param_tuning/trial_0025/ (stored 0%)
  adding: models/ride_model_param_tuning/trial